In [24]:
import numpy as np
import networkx as nx
import os

# Call this function to get the data 
def get_data(city='Amsterdam', mode='bike', root_path='../data/'):
    folders = os.listdir(root_path)
    for folder in folders:
        if folder == city:
            files = os.listdir(root_path+folder)
            bike_data = [file for file in files if mode in file][0]
            path = root_path + folder+ '/'+ bike_data;
    return nx.read_graphml(path)

G = get_data()
print(type(G))

<class 'networkx.classes.multidigraph.MultiDiGraph'>


In [25]:
# for n, d in list(G.nodes(data=True)):
#     print(d)
n='6316199'
G.nodes[n]['x']

'628561.4884115539'

In [26]:
# print(G.edges(('6316199', '1360288038'), data=True))

for u, v, d in list(G.edges(data=True)):
    d['length'] = float(d['length'])

G.edges(('6316199', '1360288038'), data=True)

OutMultiEdgeDataView([('6316199', '1360288038', {'osmid': '4745774', 'oneway': 'False', 'name': 'Raamsteeg', 'highway': 'cycleway', 'width': '1.80', 'length': 5.164}), ('1360288038', '6316199', {'osmid': '4745774', 'oneway': 'False', 'name': 'Raamsteeg', 'highway': 'cycleway', 'width': '1.80', 'length': 5.164}), ('1360288038', '1069444689', {'osmid': '4745774', 'oneway': 'False', 'name': 'Raamsteeg', 'highway': 'cycleway', 'width': '1.80', 'length': 3.831})])

In [27]:
def pick_random(G, num=2):
    nodes = np.random.choice(G.nodes(), num)
    return nodes

def compute_path_lengths(num=1000):
    failed = 0
    path_length = 0
    for i in range(num): 
        u,v = pick_random(G)
        try: 
            path_length += nx.shortest_path_length(G, source=u, target=v, weight='length')
        except:
            failed += 1
    return failed/num, path_length
        

In [30]:
num = len(G.nodes()) // 10
failed, path_length = compute_path_lengths(num)
print(failed, path_length/(num*(1-failed)))


0.5866164542294322 8140.240419761739


In [33]:
def new_edge_length(G, u=None, v=None):
    """
        given two nodes in graph G, return the straight line distance (as the crow flies) between them
    """
    if not u and not v:
        # randomly choose u and v
        pass
        u, v = np.random.choice(G.nodes(), 2)
        while v in G[u]:
            u,v = np.random.choice(G.nodes(), 2)
            
    elif not u and v:
        # randomly choose u
        pass
        u = pick_random(G, 1)
        while v in G[u]:
            u = np.random.choice(G.nodes(), 1)
            
    elif u and not v:
        pass
        # randomly choose v
        v = pick_random(G, 1)
        while v in G[u]:
            u = np.random.choice(G.nodes(), 1)
    
    xdist = float(G.nodes[u]['x']) - float(G.nodes[v]['x'])
    ydist = float(G.nodes[u]['y']) - float(G.nodes[v]['y'])
    length = (xdist**2 + ydist**2)**0.5
    return length

def calc_directness(G, u, v):
    """
        Calculate the directness between two nodes u and v
        Directness is the ratio between:
            -the shortest node path length (number of edges times the length of each edge)
            - the straight line distance between those two nodes
    """
    # NEEDS WORK
    path_length = nx.shortest_path_length(G, source=u, target=v, weight='length')
    direct_length = new_edge_length(G, u, v)
    return direct_length/path_length


In [53]:
wcc = [cc for cc in nx.weakly_connected_component_subgraphs(G)]
wcc.sort(key=len, reverse=True)
print(len(wcc))
print(len(wcc[0]))
G2 = wcc[0]
d = 0
# pick 1000 random edges from the largest component and calculate the average directness d (NEEDS WORK)
for i in range(1000):
    c = np.random.choice(len(G2.edges()), 1)
    edges = list(G2.edges())
    u, v = edges[int(c)]
    d += (calc_directness(G2, u, v) / 1000)

d

355
23781


In [54]:
d

1.0017909835154841